### Install required libraries

In [15]:
%pip install selenium
%pip install google-api-python-client==1.7.2 google-auth==1.8.0 google-auth-httplib2==0.0.3 google-auth-oauthlib==0.4.1
%pip install youtube-transcript-api
%pip install cohere

DEPRECATION: Loading egg at /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyaml-23.5.8-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/docker-6.1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: Loading egg at /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyaml-23.5.8-py3.11.egg is deprecated. pip 24.3 will enforce this b

### Setting up youtube api

In [71]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Set up the OAuth 2.0 flow
flow = InstalledAppFlow.from_client_secrets_file(
    'client_secrets.json',
    scopes=['https://www.googleapis.com/auth/youtube.readonly']
)

# Run the flow and retrieve credentials
credentials = flow.run_console()

# Build the YouTube service
youtube = build('youtube', 'v3', credentials=credentials)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=649026427795-utntt3ka70k90578lt3ch59j21oi1i80.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=T67wgkH75nxRtP8fvaPX7jd8bTeoNE&prompt=consent&access_type=offline


ValueError: Please supply either code or authorization_response parameters.

### Getting subscription list

In [63]:
from youtube_transcript_api import YouTubeTranscriptApi

# Make a request to get the recommended videos
request = youtube.subscriptions().list(
    maxResults=20,
    part='snippet',
    mine=True,
)

#get channel ids
response = request.execute()
channel_ids = [item['snippet']['resourceId']['channelId'] for item in response['items']]
print(channel_ids)
print(len(channel_ids))

['UCXUPKJO5MZQN11PqgIvyuvQ', 'UCBGxe5vTiVWv7e2gP6QB1Mw', 'UCxladMszXan-jfgzyeIMyvw', 'UCMLtBahI5DMrt0NPvDSoIRQ', 'UCNJ1Ymd5yFuUPtn21xtRbbw', 'UCbfYPyITQ-7l4upoX8nvctg', 'UCtYLUTtgS3k1Fg4y5tAhLbw', 'UCcQx1UnmorvmSEZef4X7-6g', 'UCCcrR0XBH0aWbdffktUBEdw', 'UCgBncpylJ1kiVaPyP-PZauQ', 'UCfzlCWGWYyIQ0aLC5w48gBQ', 'UC2SAJYajB1fKUXVXGKoJslQ', 'UCHnyfMqiRRG1u-2MsSQLbXA', 'UCnVzApLJE2ljPZSeQylSEyg', 'UC58v9cLitc8VaCjrcKyAbrw', 'UCR1-GEpyOPzT2AO4D_eifdw', 'UCr8O8l5cCX85Oem1d18EezQ', 'UCJINtWke3-FMz2WuEltWDVQ', 'UCX7Y2qWriXpqocG97SFW2OQ', 'UCWN3xxRkmTPmbKwht9FuE5A']
20


### Getting transcripts of youtube videos

In [72]:
def get_transcript(video_id):
    try:
        transcript_parts = YouTubeTranscriptApi.get_transcript(video_id)
        transcript = ''
        for part in transcript_parts:
            transcript += part['text'] + ' '
        return transcript
    except:
        return ''

videos = []
for channel_id in channel_ids:
    request = youtube.search().list(
        part='snippet',
        channelId=channel_id,
        type='video',
        maxResults=1
    )
    response = request.execute()
    videos.extend(response['items'])

video_urls = ['https://www.youtube.com/watch?v=' + video['id']['videoId'] for video in videos]
video_ids = [video['id']['videoId'] for video in videos]
video_thumbnails = [video['snippet']['thumbnails']['default']['url'] for video in videos]
video_author = [video['snippet']['channelTitle'] for video in videos]


transcripts = [get_transcript(video_id) for video_id in video_ids]
print(len(transcripts))

20


### Creating decorator for exponential backoff for Co:here API

In [74]:
import time
from functools import wraps

def retry_with_exponential_backoff(retries=3, initial_delay=2):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            attempt = 0
            delay = initial_delay
            while attempt < retries:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f"Attempt {attempt + 1} failed: {e}")
                    attempt += 1
                    time.sleep(delay)
                    delay *= 2  # Exponential backoff
            return ''
        return wrapper
    return decorator

### Creating Cohere client and obtaining bullet points

In [76]:
import cohere
from dotenv import load_dotenv
import os

load_dotenv(".env")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
co = cohere.Client(COHERE_API_KEY)

@retry_with_exponential_backoff(retries=10, initial_delay=2)
def get_bullet_points(transcript):
    response = co.chat(
        model="command-r-plus",
        message="Give me three takeaway bullet points of this video given the following transcript. Format the bullet points like so: -bullet point. Do not include anything else: " + transcript,
    )
    print(response.text)
    return response.text

def extract_bullet_points(response):
    bullet_points = response.split('\n')
    bullet_points = [point for point in bullet_points if point != '' and point[0] == '-']
    return bullet_points

bullet_points = [get_bullet_points(transcript) for transcript in transcripts]
bullet_points_filtered = [extract_bullet_points(summary) for summary in bullet_points if summary != '']
print(bullet_points_filtered)
print(len(bullet_points_filtered))

- Bicycles can stay upright without a rider as long as they are moving with sufficient speed.

- It is not the gyroscopic effect of the spinning wheel that keeps a bike upright, but rather the self-correcting steering design that brings the wheels back underneath the bike if it starts to fall. 

- Locking the handlebars of a moving bike to prevent turning will cause it to fall, demonstrating the importance of the self-steering mechanism in maintaining stability.
- Large Language Models (LLMs) are trained on massive amounts of text data, compressed into neural networks, and used for various tasks like text and image generation.
- LLMs are evolving to use tools and existing computing infrastructure, becoming more capable problem solvers, and are expected to improve further with time.
- Security challenges, such as jailbreak, prompt injection, and data poisoning attacks, are emerging in the new computing paradigm, requiring ongoing defense strategies.
- μTransfer optimizes hyperparameters

### Getting a quick one line summary of the video

In [66]:
@retry_with_exponential_backoff(retries=10, initial_delay=2)
def get_header(summary):
    response = co.chat(
        model="command-r-plus",
        message="Give me a one line title for a video that is summarized as the following. Only give me the title, nothing else: " + summary,
    )
    print(response)
    return response.text

headers = [get_header(summary) for summary in bullet_points]
headers_filtered = [header for header in headers if header != '']
print(headers_filtered)
print(len(headers_filtered))


Attempt 1 failed: status_code: 429, body: data=None
text='"Understanding the \'axis\' Parameter in Pandas: Rows vs Columns."' generation_id='d0afd04a-313a-4ae8-937e-ae796c95a3b1' citations=None documents=None is_search_required=None search_queries=None search_results=None finish_reason='COMPLETE' tool_calls=None chat_history=[Message_User(message="Give me a one line title for a video that is summarized as the following. Only give me the title, nothing else: - The `axis` parameter in pandas specifies whether an operation should be applied to rows or columns, with `axis=0` referring to rows (index) and `axis=1` referring to columns.\n- By default, the `mean` function in pandas calculates the mean of each column (axis=0), but you can specify `axis=1` to calculate the mean of each row.\n- Pandas allows you to use aliases for the axis numbers, such as `'index'` for `axis=0` and `'columns'` for `axis=1`, to make the code more intuitive and readable.", tool_calls=None, role='USER'), Message_C

### Getting tags for the video

In [67]:
@retry_with_exponential_backoff(retries=10, initial_delay=2)
def get_tags(summary):
    response = co.chat(
        model="command-r-plus",
        message="Give me 3 tags that are in the format #tag for a video that is summarized as the following. Only give me the tags nothing else: " + summary,
    )
    print(response)
    return response.text

def filter_tags(response):
    tags = response.split()
    filtered_tags = [tag for tag in tags if tag[0] == '#']
    return filtered_tags

tags = [get_tags(summary) for summary in bullet_points]
tags_filtered = [filter_tags(tag) for tag in tags if tag != '']

print(tags_filtered)
print(len(tags_filtered))

Attempt 1 failed: status_code: 429, body: data=None
Attempt 2 failed: status_code: 429, body: data=None
Attempt 3 failed: status_code: 429, body: data=None
Attempt 4 failed: status_code: 429, body: data=None
Attempt 5 failed: status_code: 429, body: data=None
text='1. #pandas\n2. #data-analysis\n3. #pandas-axis' generation_id='f9bac39f-2ef2-476a-8727-52fb6218f754' citations=None documents=None is_search_required=None search_queries=None search_results=None finish_reason='COMPLETE' tool_calls=None chat_history=[Message_User(message="Give me 3 tags that are in the format #tag for a video that is summarized as the following. Only give me the tags nothing else: - The `axis` parameter in pandas specifies whether an operation should be applied to rows or columns, with `axis=0` referring to rows (index) and `axis=1` referring to columns.\n- By default, the `mean` function in pandas calculates the mean of each column (axis=0), but you can specify `axis=1` to calculate the mean of each row.\n- 

### Creating Email format

In [69]:
import smtplib
from email.mime.text import MIMEText
from datetime import datetime
import os

EMAIL_PASSWORD = os.environ.get("EMAIL_PASSWORD")
EMAIL = os.environ.get("EMAIL")

def format_email_text(headers, bullet_points, tags, video_urls, video_ids, authors):
    html = """
                <h1>🤩 Here is a list of some recommended videos for you! 🤩</h1>
                <hr/>
            """
    for i in range(len(headers)):
        bullet_points_html = '<ul>'
        for bullet in bullet_points[i]:
            bullet_points_html += f"<li>{bullet[1:]}</li>"
        bullet_points_html += '</ul>'
        html += f"""
        <div style="margin-bottom: 20px;">
            <h2>{headers[i]}</h2>
            <p><strong>Author:</strong> {authors[i]}</p>
            <a href="{video_urls[i]}" target="_blank">
                <img src="https://img.youtube.com/vi/{video_ids[i]}/maxresdefault.jpg" style="width: 640px; height: 360px;" />
            </a>
            <p><strong>Summary:</strong> {bullet_points_html}</p>
            <p><strong>Tags:</strong> <span style="color: blue;">{' '.join(tags[i])}</span></p>
        </div>
        """
    html += """
                <hr/>
                <p>🚀🚀🚀</p>
                <p>This email was powered by Auto-Newsletter</p>
            """
    return html


def get_ordinal_suffix(day):
    if 11 <= day <= 13:
        return "th"
    last_digit = day % 10
    if last_digit == 1:
        return "st"
    elif last_digit == 2:
        return "nd"
    elif last_digit == 3:
        return "rd"
    else:
        return "th"

# Define your email credentials and SMTP server details
smtp_server = 'smtp.gmail.com'
smtp_port = 587
email_user = EMAIL
email_password = EMAIL_PASSWORD

now = datetime.now()
day = now.day
month = now.strftime("%B")
year = now.year
suffix = get_ordinal_suffix(day)
formatted_date = f"{month} {day}{suffix}, {year}"

custom_text = format_email_text(headers_filtered, bullet_points_filtered, tags_filtered, video_urls, video_ids, video_author)

# Create the MIMEText object
msg = MIMEText(custom_text, 'html')
msg['Subject'] = f'🚀 Your Auto-Newsletter is Ready! {formatted_date} 🚀'
msg['From'] = email_user
msg['To'] = email_user

print(custom_text)


                <h1>🤩 Here is a list of some recommended videos for you! 🤩</h1>
                <hr/>
            
        <div style="margin-bottom: 20px;">
            <h2>"Understanding the 'axis' Parameter in Pandas: Rows vs Columns."</h2>
            <p><strong>Author:</strong> Data School</p>
            <a href="https://www.youtube.com/watch?v=PtO3t6ynH-8" target="_blank">
                <img src="https://img.youtube.com/vi/PtO3t6ynH-8/maxresdefault.jpg" style="width: 640px; height: 360px;" />
            </a>
            <p><strong>Summary:</strong> <ul><li> The `axis` parameter in pandas specifies whether an operation should be applied to rows or columns, with `axis=0` referring to rows (index) and `axis=1` referring to columns.</li><li> By default, the `mean` function in pandas calculates the mean of each column (axis=0), but you can specify `axis=1` to calculate the mean of each row.</li><li> Pandas allows you to use aliases for the axis numbers, such as `'index'` for `axi

### Sending email

In [70]:
# Set up the server and send the email
try:
    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()  # Upgrade the connection to a secure encrypted SSL/TLS connection
    server.login(email_user, email_password)
    server.sendmail(email_user, email_user, msg.as_string())
    server.quit()
    print("Email sent successfully!")
except Exception as e:
    print(f"Failed to send email: {e}")

Connecting to the SMTP server...
Logging in...
Sending email...
Email sent successfully!
